Portfolio Construction

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels as sm
import matplotlib as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (16,8)

Populating the interactive namespace from numpy and matplotlib


/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [60]:
dfqs = pd.read_csv('//Users//Jamie//Documents//Jupyter Notebook Files//quant_scores.csv', index_col = 0, parse_dates=True)
dftr = pd.read_csv('//Users//Jamie//Documents//Jupyter Notebook Files//total_returns.csv', index_col = 0, parse_dates=True)
dfindwt = pd.read_csv('//Users//Jamie//Documents//Jupyter Notebook Files//index_wts.csv', index_col = 0, parse_dates=True)
dfpsei = pd.read_csv('//Users//Jamie//Documents//Jupyter Notebook Files//pcomp_return.csv', index_col = 0, parse_dates=True)

In [61]:
dfqs.head(5)

,BCI,ION,ICT,URC,UBP,SMCB,ABS,FLI,RLC,EEI,...,RCB,VLL,GMA7,PSE,CHIB,CEB,SMDC,BLOOM,LTG,EMP
36922,NaN,-0.062443,-0.053796,0.307173,0.196642,0.516090,-0.406294,-0.482474,NaN,NaN,...,NaN,0.307173,NaN,NaN,NaN,NaN,0.516090,NaN,NaN,NaN
36950,NaN,-0.035469,-0.050858,0.196175,0.275168,-0.364142,-0.229654,-0.417662,NaN,NaN,...,NaN,0.196175,NaN,NaN,NaN,NaN,-0.364142,NaN,NaN,NaN
36981,NaN,0.258605,0.328888,-0.095036,-0.132697,0.055743,0.147378,0.116802,NaN,NaN,...,NaN,-0.095036,NaN,NaN,NaN,NaN,0.055743,NaN,NaN,NaN
37011,NaN,0.382619,0.074050,-0.842184,-0.392563,0.369888,0.301919,0.543440,NaN,NaN,...,NaN,-0.842184,NaN,NaN,NaN,NaN,0.369888,NaN,NaN,NaN
37042,NaN,-0.540034,0.028212,0.086328,0.624103,0.287636,0.212853,-0.094265,NaN,NaN,...,NaN,0.086328,NaN,NaN,NaN,NaN,0.287636,NaN,NaN,NaN


In [76]:
#overweight/underweight rules
def OW_UW(qs, s):
    if np.isnan(qs):
        return 0
    elif (qs<-1):
        return s*0
    elif (qs>=-1) & (qs<-0.75):
        return s*-0.25
    elif (qs>=-0.75) & (qs<-0.5):
        return s*-0.5
    elif (qs>=-0.5) & (qs<-0.25):
        return s*-0.5
    elif (qs>=-0.25) & (qs<0):
        return s*-0.5
    elif (qs>=0) & (qs<0.25):
        return s*0
    elif (qs>=0.25) & (qs<0.5):
        return s*0.25
    elif (qs>=0.5) & (qs<0.75):
        return s*0*5
    elif (qs>=0.75) & (qs<1):
        return s*0.5
    elif (qs>=1) & (qs<1.25):
        return s*1
    elif (qs>=1.25) & (qs<1.5):
        return s*1
    elif (qs>=1.5):
        return s*0.5

In [84]:
#creating normalized index weights based on qs
dfpwt = dfindwt
tlt = 1 #tilt max
x, y = dfpwt.shape
for a in range(0,x):
    for b in range(0,y):
        if dfindwt.loc[dfindwt.index[a], dfindwt.columns[b]]==0:
            dfpwt.loc[dfpwt.index[a], dfpwt.columns[b]]=0
        elif dfindwt.loc[dfindwt.index[a], dfindwt.columns[b]]>0:
            dfpwt.loc[dfpwt.index[a], dfpwt.columns[b]] = \
            max(OW_UW(dfqs.loc[dfqs.index[a], dfqs.columns[b]], tlt)/100 \
                + dfindwt.loc[dfpwt.index[a], dfindwt.columns[b]], 0)

dfpwt.head(5)

,BCI,ION,ICT,URC,UBP,SMCB,ABS,FLI,RLC,EEI,...,RCB,VLL,GMA7,PSE,CHIB,CEB,SMDC,BLOOM,LTG,EMP
36922,0,0.000000,0.000000,0.0,0.0,0.040741,0.008921,0.000000,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36950,0,0.000000,0.000000,0.0,0.0,0.024673,0.019924,0.000000,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36981,0,0.012631,0.009981,0.0,0.0,0.037337,0.034002,0.008427,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37011,0,0.012881,0.002791,0.0,0.0,0.044672,0.037413,0.008469,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37042,0,0.000000,0.005355,0.0,0.0,0.046119,0.031105,0.000000,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
#normalization
#x, y = dfpwt.shape
dfsum = pd.DataFrame(dfpwt.sum(axis=1), columns=['Sum'])
dfpwt = dfpwt.divide(dfsum['Sum'], axis=0)
dfpwt.head(5)

,BCI,ION,ICT,URC,UBP,SMCB,ABS,FLI,RLC,EEI,...,RCB,VLL,GMA7,PSE,CHIB,CEB,SMDC,BLOOM,LTG,EMP
36922,0.0,0.000000,0.000000,0.0,0.0,0.048098,0.010533,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36950,0.0,0.000000,0.000000,0.0,0.0,0.028374,0.022913,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36981,0.0,0.012931,0.010218,0.0,0.0,0.038221,0.034807,0.008627,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37011,0.0,0.012891,0.002793,0.0,0.0,0.044706,0.037442,0.008476,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37042,0.0,0.000000,0.005665,0.0,0.0,0.048791,0.032907,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#create drifted portfolio, given rebalancing frequency
